In [ ]:
import numpy as np
import math
from scipy.optimize import fsolve
import pvlib

In [ ]:
Vmp = 36.6
Imp = 8.20
Voc = 45.3
Isc= 8.84 
Ncell = 72

In [ ]:
k = 1.38 * (10**-23)
q = 1.6 * (10**-19)
T = 298.15 # Temp in kelvin
Vt = (k * T) / q
print ("Vt=",Vt)

# Q1

In [ ]:
def eta_initial():
    n1 = 2*Vmp - Voc
    n2 = Ncell * Vt
    n3 = np.log((Isc - Imp) / Isc)
    n4 = Imp / (Isc - Imp)
    n5 = n1 / (n2 * (n3 + n4))
    #print ( "inieta = ", n5)
    return n5

In [ ]:
eta_initial()
eta_ini = eta_initial()
#x= int(ini1)
print (eta_ini)
#print (x)

In [ ]:
def Rs_initial():
    n1 = Vmp/Imp
    n2 = (2*Vmp-Voc)/(Isc-Imp)
    n3 = (np.log((Isc-Imp)/Isc)+(Imp/(Isc-Imp)))
    n4 = n2/n3
    n5 = n1-n4
    return n5

In [ ]:
Rs_initial()
Rs_ini=Rs_initial()
print ("Rs_initial = ",Rs_ini,"Ohms")

In [ ]:
def Rsh_initial():
    n1 = Isc/(eta_ini*Ncell*Vt)
    #print (n1)
    n2 = (Rs_ini*Isc-Voc)/(eta_ini*Ncell*Vt)
    #print (n2)
    n3 = np.exp(n2)
    #print (n3)
    n4 = n1*n3
    n5 = Rs_ini/n4
    #print (n5)
    n6 = np.sqrt(n5)
    return n6

In [ ]:
Rsh_initial()
Rsh_ini=Rsh_initial()
print ("Rsh_initial = ",Rsh_ini,"Ohms")

# Q2

In [ ]:
def parameter_function(x):
    eta = x[0]
    Rs  = x[1]
    Rsh = x[2]
    F=np.zeros(np.size(x))
    F[0] = Imp-Isc+(Vmp+Imp*Rs-Isc*Rs)/Rsh+(Isc-(Voc-Isc*Rs)/Rsh )*(np.exp((-Voc+Vmp+Imp*Rs)/(eta*Ncell*Vt ))) 
    F[1] = Imp+Vmp*(((-(-Voc+Isc*Rsh+Isc*Rs)/(eta*Ncell*Vt*Rsh ))* (np.exp((-Voc+Vmp+Imp*Rs)/(eta*Ncell*Vt )))-1/Rsh) /(1+((-Voc+Isc*Rsh+Isc*Rs)/(eta*Ncell*Vt*Rsh))*Rs*(np.exp((-Voc+Vmp+Imp*Rs)/(eta*Ncell*Vt )))+Rs/Rsh ))
    F[2] = 1/Rsh+(((-(-Voc+Isc*Rsh+Isc*Rs)/(eta*Ncell*Vt*Rsh ))* (np.exp((-Voc+Isc*Rs)/(eta*Ncell*Vt )))-1/Rsh) /(1+((-Voc+Isc*Rsh+Isc*Rs)/(eta*Ncell*Vt*Rsh))*Rs*(np.exp((-Voc+Isc*Rs)/(eta*Ncell*Vt )))+Rs/Rsh ))
    return F

x_ini = np.array([eta_ini,Rs_ini,Rsh_ini])
x = fsolve(parameter_function,x_ini)
print (x)

In [ ]:
eta = x[0]
Rs  = x[1]
Rsh = x[2]
print("eta=",eta)
print("Rs=",Rs)
print("Rsh=",Rsh)

In [ ]:
def I_saturation():
    n1=(Voc-Isc*Rs)/Rsh
    n2=Isc-n1
    #print(n2)
    n3=np.exp(-Voc/(eta*Ncell*Vt))
    #print(n3)
    n4=n2*n3
    return n4

In [ ]:
I_saturation()
Isat=I_saturation()
print ("Isat  = ",Isat,"Amps")

In [ ]:
def I_photon():
    n1=np.exp(Voc/(eta*Ncell*Vt))
    #print(n1))
    n2=Isat*n1
    #print(n2)
    n3=Voc/Rsh
    #print(n3)
    n4=n2+n3
    return n4

In [ ]:
I_photon()
Iph=I_photon()
print ("Iph = ",Iph,"Amps")

# Q3

In [ ]:
def V_oc(x):
    Voc=x[0]
    F=np.zeros(np.size(x))
    F[0]= Iph-Isat*(np.exp(Voc/(eta*Ncell*Vt)))-Voc/Rsh 
    return F

x=fsolve(V_oc,72*0.6)
Voc=x[0]
print("Voc=",Voc)

In [ ]:
i=[]

def I_sc(x):
    I=x[0]
    F=np.zeros(np.size(x))
    F[0]= -I + Iph - Isat*(np.exp((V+I*Rs) / (eta*Ncell*Vt))-1) - (V+I*Rs)/Rsh
    return F

In [ ]:
Vx=np.linspace(0,Voc,num=256)
for V in Vx:
    x = fsolve(I_sc,Iph)
    i.extend(x)
    
print("I=",i)
print("V=",Vx)

In [ ]:
import matplotlib.pyplot as plt 

In [ ]:
plt.plot(Vx, i)
plt.title('3.IV Curve at STC')
plt.xlabel('V in Volts')
plt.ylabel('I in Amperes')
plt.axis([0, 50, 0, 10])
plt.annotate('Voc',xy=(45.3,0),xytext=(45.3,-0.5))
plt.annotate('Isc',xy=(0,8.839999758809823),xytext=(-2,8.839999758809823))
plt.annotate('Pmax',xy=(36.6040393,8.201053667922631))

In [ ]:
p=[]
p=np.multiply(Vx,i)
print("P=",p)

In [ ]:
plt.plot(Vx, p)
plt.axis([0, 50, 0, 310])
plt.title('3.Power Curve at STC')
plt.xlabel('V in Volts')
plt.ylabel('Power in Watts')
plt.show

In [ ]:
Voc=np.amax(Vx)
Isc=np.amax(i)
Pmax=np.amax(p)
print("Voc=",Voc)
print("Isc=",Isc)
print("Pmax=",Pmax)
print("Rs=",Rs)
print("Rsh=",Rsh)
print("Iph=",Iph)
print("Isat=",Isat)

In [ ]:
FF=(Pmax/(Voc*Isc))
print("Fill Factor=",FF)

# Q4

In [ ]:
Rs=2*Rs
print("Rs1=",Rs)

In [ ]:
x=fsolve(V_oc,72*0.6)
Voc1=x[0]
print("Voc=",Voc1)

In [ ]:
i1=[]
Vx=np.linspace(0,Voc1,num=256)
for V in Vx:
    x = fsolve(I_sc,Iph)
    i1.extend(x)
    
print("I=",i1)
print("V=",Vx)

In [ ]:
markers_on = [206]
plt.plot(Vx, i1,label='Rs1')
plt.plot(Vx, i,label='Rs')
plt.legend()
plt.title('4.IV Curve at (2*Rs)')
plt.xlabel('V in Volts')
plt.ylabel('I in Amperes')
plt.axis([0, 50, 0, 10])
plt.annotate('Voc',xy=(45.3,0),xytext=(45.3,-0.5))
plt.annotate('Isc',xy=(0,8.839999758809823),xytext=(-2,8.839999758809823))
plt.annotate('Pmax',xy=(36.6040393,8.201053667922631))
plt.show()

In [ ]:
p1=[]
p1=np.multiply(Vx,i1)
print("P=",p1)


In [ ]:
Voc1=np.amax(Vx)
Isc1=np.amax(i1)
Pmax1=np.amax(p1)
print("Voc=",Voc1)
print("Isc=",Isc1)
print("Pmax=",Pmax1)
print("Rs=",Rs)
print("Rsh=",Rsh)
print("Iph=",Iph)
print("Isat=",Isat)
FF1=(Pmax1/(Voc1*Isc1))
print("Fill Factor=",FF1)
index=np.where(p1==np.amax(p1))
print(index)
print("point=",Vx[index])
print("point1=",i1[206])

# Q4 % degradation

In [ ]:
deltaPmax=((Pmax-Pmax1)/Pmax)
print ("deltaPmax=",deltaPmax*100,"%")

In [ ]:
deltaFF=((FF-FF1)/FF)
print ("deltaFF=",deltaFF*100,"%")

# Q5

In [ ]:
Rs=0.21721509749730586 # Q3 as baseline
Rsh2=np.linspace(50,Rsh,num=10)
print("Rsh2=",Rsh2)


In [ ]:
voc=[]
for Rsh in Rsh2:
    x=fsolve(V_oc,72*0.6)
    voc.append(x)

print("voc=",voc)
    

In [ ]:
i2=[]
p2=[]
Isc_2=[]
Voc_2=[]
Pmax_2=[]
FF_2=[]
Vx=np.linspace(0,Voc,num=256)
for Rsh in Rsh2:
    for V in Vx:
        x = fsolve(I_sc,Iph)
        i2.extend(x)
    y=0
    Voc2=np.amax(Vx)
    #print("Voc=",Voc2)
    Isc2=np.amax(i2)
    #print("Isc=",Isc2)
    p2=np.multiply(Vx,i2)
    y=p2.size
    print(y)
    Pmax2=np.amax(p2)
    #print("Pmax=",Pmax2)
    FF2=(Pmax2/(Voc2*Isc2))
    #print("FF=",FF2)
    Isc_2.append(Isc2)
    Voc_2.append(Voc2)
    Pmax_2.append(Pmax2)
    FF_2.append(FF2)
    i2.clear()
print("Isc_2=",Isc_2)
print("Pmax_2=",Pmax_2)
print("FF_2=",FF_2)

In [ ]:
Voc_delta=[]
Isc_delta=[]
Pmax_delta=[]
FF_delta=[]
for i in range(10):
    Vocdelta=((Voc-voc[i])/Voc)*100
    Iscdelta=((Isc-Isc_2[i])/Isc)*100
    Pmaxdelta=((Pmax-Pmax_2[i])/Pmax)*100
    FFdelta=((FF-FF_2[i])/FF)*100
    Voc_delta.extend(Vocdelta)
    Isc_delta.append(Iscdelta)
    Pmax_delta.append(Pmaxdelta)
    FF_delta.append(FFdelta)

print(Voc_delta)
print(Isc_delta)
print(Pmax_delta)
print(FF_delta)

In [ ]:
plt.plot(Rsh2, Pmax_delta)
plt.axis([0, 1200, 0, 10])
plt.title('5.Pmax_delta % Vs Rsh')
plt.ylabel('Pmax_delta % in Watts')
plt.xlabel('R in ohms')

In [ ]:
plt.plot(Rsh2, FF_delta)
plt.axis([0, 1200, 0, 10])
plt.title('5.FF_delta % Vs Rsh')
plt.ylabel('FF_delta %')
plt.xlabel('R in ohms')

In [ ]:
plt.plot(Rsh2, Voc_delta)
plt.axis([0, 1200, 0, 0.8])
plt.title('5.Voc_delta % Vs Rsh')
plt.ylabel('Voc_delta % in Volts')
plt.xlabel('R in ohms')

In [ ]:
plt.plot(Rsh2, Isc_delta)
plt.axis([0, 1200, 0, 0.5])
plt.title('5.Isc_delta % Vs Rsh')
plt.ylabel('Isc_delta % in Amps')
plt.xlabel('R in ohms')

# Q5.2

In [ ]:
Rsh3= 10*Rsh
print(Rsh3)

In [ ]:
Rsh = Rsh3
x=fsolve(V_oc,72*0.6)
Voc=x[0]
print("Voc=",Voc)

In [ ]:
Rs=0.21721509749730586
i3=[]
Vx=np.linspace(0,Voc,num=256)
for V in Vx:
    x = fsolve(I_sc,Iph)
    i3.extend(x)

In [ ]:
p3=[]
p3=np.multiply(Vx,i3)
Pmax3=np.amax(p3)
print("Pmax3=",Pmax3)


In [ ]:
P_delta=[]
for i in range(256):
    Pdelta=(p3[i]-p[i])
    P_delta.append(Pdelta)
    

In [ ]:
plt.plot(Vx,p)
plt.axis([0, 50, 0, 310])
plt.title('5.Power(Rsh) Curve')
plt.ylabel('Power in watts')
plt.xlabel("V in Volts")
plt.show()

plt.plot(Vx,p3)
plt.axis([0, 50, 0, 310])
plt.title('5.Power(10*Rsh) Curve')
plt.ylabel('Power in watts')
plt.xlabel("V in Volts")
plt.show()
